In [ ]:
from typing import Tuple, List

from src.material.textures.material.checker import CheckerMaterial
from src.material.textures.noise.simplex_noise import SimplexNoise
from src.material.textures.noise.voronoi_noise import VoronoiNoise
from src.render.helpers import cast_ray
from src.render.loops import *
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig
from src.material.material.phong_material import PhongMaterial
from src.material.color import Color
from src.geometry.primitives.sphere import Sphere
from src.scene.primitive import Object
from src.scene.scene import Scene
from src.scene.camera import Camera
from src.scene.light import PointLight
from src.math import Vertex, Vector
from src.render.resolution import Resolution
from src.scene.light import SpotLight
from src.geometry.primitives.box import Box
from src.material.textures.material.marble import MarbleMaterial
from src.material.textures.noise.ridget_multifractal import RidgeNoise
from src.material.textures.noise.turbulence_noise import TurbulenceNoise
from src import NormalShader, AmbientLight, ipynb_display_images
from src.material.textures.noise.perlin_noise import PerlinNoise
from src.material.textures.noise.FBM import FBMNoise
from src.material.textures.material.rock import RockMaterial
from src.scene import primitive
from dataclasses import dataclass
from src.render.loops.render_loop import RenderLoop

In [ ]:

from src.geometry.primitives.box import Box
from src.material.color import to_u8


glossy_blue = PhongMaterial(
    name="glossy_blue",
    base_color=Color.custom_rgb(30, 80, 255),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=120,
)

glossy_green = PhongMaterial(
    name="glossy_green",
    base_color=Color.custom_rgb(30, 255, 80),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=120,
)





scene = Scene(
    camera=Camera(
        fov=45,
        aspect_ratio=Resolution.R360p.aspect_ratio,
        origin=Vertex(0, 0, 0),
        direction=Vector(0, 0, -1)
    ),
    primitives=[
        Object(Sphere(center=Vertex(-1.5, 0.0, -5.0), radius=1.0), material=glossy_blue),
        Object(Box(corner1=Vertex(-2.0, -1.5, -4.0), corner2=Vertex(2.0, -1.4, 2.0)), material=glossy_green),
    ],
    lights=[
        PointLight(position=Vertex(5, 5, 0), intensity=1000.0),
        AmbientLight(intensity=1),

    ]
)

render_configuration = RenderConfig(
    resolution=Resolution.R144p,
    samples_per_pixel=10,
    max_depth=1,
)


# How render loops work
Simple render loops take a scene, render configuration,
and preview configuration as input, and produce an image as output.
They handle the rendering process, including sampling, ray tracing, and displaying progress.


### Lets create a simple ray casting loop

In [ ]:
@dataclass
class SimpleLoop(RenderLoop):
    def render_pixel(self, i: int, j: int) -> Tuple[int, int, int]:

        u, v = self.get_pixel_uv(i, j)

        ray = self.camera.make_ray(u, v)
        col = cast_ray(
            ray=ray,
            lights=self.lights,
            depth=self.max_depth,
            shader=self.shader,
            skybox=self.skybox,
            scene=self.scene,
        )
        return to_u8(col.r), to_u8(col.g), to_u8(col.b)

    def render_all_pixels(self):
        pixels: List[Tuple[int, int, int]] = []
        for row in range(self.height):
            for i in range(self.width):
                pixels.append(self.render_pixel(i, row))
        return pixels, self.width, self.height

    def get_pixel_uv(self, i: int, j: int) -> Tuple[float, float]:
        u = (i + 0.5) / self.width
        v = (j + 0.5) / self.height
        return u, v

# Now we can use our SimpleLoop to render the scene
- with the same configurations as before
- and see how it performs compared to the built-in LinearRayCaster

In [ ]:
my_simple_loop = SimpleLoop(scene=scene, render_config=render_configuration)
img = my_simple_loop.render("images/simple_loop.png")
ipynb_display_images(img)

my_ray_tracer = LinearRayCaster(scene=scene, render_config=render_configuration)
img = my_ray_tracer.render("images/linear_ray_caster.png")
ipynb_display_images(img)
